## Site Access Score Tool

##### May 23th, 2022

In [ ]:
# import system modules
import arcpy
from arcpy import env
import os

try:
    # Check out Network Analyst license if available. 
    # Fail if Network Analyst License is not available.
    if arcpy.CheckExtension("network") == "Available":
        arcpy.CheckOutExtension("network")
    else:
        raise arcpy.ExcuteError("Network Analyst Extension license is not available")

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"
workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
in_lat = arcpy.GetParameterAsText(0) # Double, Y
in_long = arcpy.GetParameterAsText(1) # Double, X
point_fly = arcpy.GetParameterAsText(2) # feature layer
point_buf = arcpy.GetParameterAsText(3) # feature layer
buffer_dist = arcpy.GetParameterAsText(4) # Linear unit
intsct_bufroad = arcpy.GetParameterAsText(5) # feature layer

# create a point with spatial reference
point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)

# create a point feature class
arcpy.CopyFeatures_management(point_geom, point_fly)

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_fly, point_buf, buffer_dist)

# create intersections between the buffer and major roads feature layer
  # Set local variable
major_roads = "MajorRoads"
arcpy.Intersect_analysis([point_buf, major_roads], intsct_bufroad, "", "", "point")

# Report a success message    
arcpy.AddMessage("Script completed successfully!")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

In [ ]:
# Reference - 'Origination Destination Cost Matrix'
# https://pro.arcgis.com/en/pro-app/2.8/arcpy/network-analyst/odcostmatrix.htm

# An example showing how to perform origin destination cost matrix analysis using inputs from feature classes.
import arcpy
arcpy.CheckOutExtension("network")

nds = "C:/data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origins = "C:/data/io.gdb/Origins"
input_destinations = "C:/data/io.gdb/Destinations"
output_lines = "C:/data/io.gdb/ODCostMatrixLines"

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultImpedanceCutoff = 15
odcm.defaultDestinationCount = 5
odcm.lineShapeType = arcpy.nax.LineShapeType.NoLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origins)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destinations)
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

In [ ]:
# Short
network = "C:/Data/SanFrancisco.gdb/Transportation/Streets_ND"
arcpy.na.MakeRouteAnalysisLayer(network, "InspectionRoute", "Driving Time",
                        "FIND_BEST_ORDER", "1/1/1900 9:00 AM", "UTC",
                        "ALONG_NETWORK", ["Meters", "TravelTime"])

In [ ]:
# Long
# Name: MakeRouteAnalysisLayer_Workflow.py
# Description: Find a best route to visit the stop locations and save the
#              route to a layer file. The stop locations are geocoded from a
#              text file containing the addresses.
# Requirements: Network Analyst Extension

#Import system modules
import arcpy
from arcpy import env
import os

try:
    #Check out Network Analyst license if available. Fail if the Network Analyst license is not available.
    if arcpy.CheckExtension("network") == "Available":
        arcpy.CheckOutExtension("network")
    else:
        raise arcpy.ExecuteError("Network Analyst Extension license is not available.")
    
    #Set environment settings
    output_dir = "C:/Data"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "Output.gdb")
    env.overwriteOutput = True

    #Set local variables
    input_gdb = "C:/Data/SanFrancisco.gdb"
    network = os.path.join(input_gdb, "Transportation", "Streets_ND")
    layer_name = "BestRoute"
    travel_mode = "Driving Time"
    address_locator = "C:/Data/SanFranciscoLocator"
    address_table = "C:/Data/StopAddresses.csv"
    address_fields = "Street Address;City City;State State;ZIP <None>"
    out_stops = "GeocodedStops"
    output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")

    #Create a new Route layer. For this scenario, the default values for all the
    #remaining parameters statisfy the analysis requirements
    result_object = arcpy.na.MakeRouteAnalysisLayer(network, layer_name,
                                                                    travel_mode)

    #Get the layer object from the result object. The route layer can now be
    #referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the route layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    stops_layer_name = sublayer_names["Stops"]

    #Geocode the stop locations from a csv file containing the addresses.
    #The Geocode Addresses tool can use a text or csv file as input table
    #as long as the first line in the file contains the field names.
    arcpy.geocoding.GeocodeAddresses(address_table, address_locator,
                                     address_fields, out_stops)

    #Load the geocoded address locations as stops mapping the address field from
    #geocoded stop features as Name property using field mappings.
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,
                                                            stops_layer_name)
    field_mappings["Name"].mappedFieldName = "Address"
    arcpy.na.AddLocations(layer_object, stops_layer_name, out_stops,
                            field_mappings, "")

    #Solve the route layer, ignoring any invalid locations such as those that
    #cannot be geocoded
    arcpy.na.Solve(layer_object, "SKIP")

    #Save the solved route layer as a layer file on disk
    layer_object.saveACopy(output_layer_file)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

##### May 20th, 2022

In [ ]:
# import system modules
import arcpy
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"
workspace = "C:/ArcGIS/Projects/Dev_Proto/Dev_Proto.gdb"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
in_lat = arcpy.GetParameterAsText(0) # Double, Y
in_long = arcpy.GetParameterAsText(1) # Double, X
point_fly = arcpy.GetParameterAsText(2) # feature layer
point_buf = arcpy.GetParameterAsText(3) # feature layer
buffer_dist = arcpy.GetParameterAsText(4) # Linear unit
intsct_bufroad = arcpy.GetParameterAsText(5) # feature layer

# create a point with spatial reference
point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)

# create a point feature class
arcpy.CopyFeatures_management(point_geom, point_fly)

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_fly, point_buf, buffer_dist)

# create intersections between the buffer and major roads feature layer
  # Set local variable
major_roads = "MajorRoads"
arcpy.Intersect_analysis([point_buf, major_roads], intsct_bufroad, "", "", "point")

# Report a success message    
arcpy.AddMessage("All done!")


##### May 13th, 2022

In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True
arcpy.env.qualifiedFieldNames = False 
   # Output field name includes the table name

# set parameters
in_table = arcpy.GetParameterAsText(0) # input table view
in_field = arcpy.GetParameterAsText(1) # input table field
join_table = arcpy.GetParameterAsText(2) # join table
join_field = arcpy.GetParameterAsText(3) # join table field
out_point = arcpy.GetParameterAsText(4) # feature class

# table join with PopCenterZip
in_join_table = arcpy.AddJoin_management(in_table, in_field, 
                                        join_table, join_field)

# create a point from table with spatial reference
arcpy.management.XYTableToPoint(in_join_table, out_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))


In [ ]:
# import system modules
import arcpy
import os
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True
arcpy.env.qualifiedFieldNames = False 
   # Output field name includes the table name

# set parameters
in_table = arcpy.GetParameterAsText(0) # table view
in_field = arcpy.GetParameterAsText(1) # field
join_table = "C:\ArcGIS\Projects\Dev_Proto\Dev_Proto.gdb\PopCenterZips_Table" # join table
join_field = "ZipCode"
out_point = arcpy.GetParameterAsText(2) # feature class

# table join with PopCenterZip
in_join_table = arcpy.AddJoin_management(in_table, in_field, 
                                        join_table, join_field)

# create a point from table with spatial reference
arcpy.management.XYTableToPoint(in_join_table, out_point,
                               "Long", "Lat",
                               arcpy.SpatialReference("WGS 1984"))


In [ ]:
# Please try to use in memory 
# and delete memory

##### May 10th, 2022

In [ ]:
# import system modules
import arcpy
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
in_lat = arcpy.GetParameterAsText(0) # Double, Y
in_long = arcpy.GetParameterAsText(1) # Double, X
point_shp = arcpy.GetParameterAsText(2) # feature class
point_buf = arcpy.GetParameterAsText(3) # feature class
point_buf_fl = arcpy.GetParameterAsText(4) # feature layer
intsc_buf_mr = arcpy.GetParameterAsText(5) # feature layer
buffer_dist = arcpy.GetParameterAsText(6) # Linear unit

# create a point with spatial reference
point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)

# create a point feature class
arcpy.CopyFeatures_management(point_geom, point_shp)

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_shp, point_buf, buffer_dist)

# create a feature layer with the buffer
arcpy.MakeFeatureLayer_management(point_buf, point_buf_fl)

# create intersections between the buffer and major roads feature layer
arcpy.Intersect_analysis([point_buf_fl, "C:\ArcGIS\Projects\Dev_Proto\Dev_Proto.gdb\MajorRoads"], intsc_buf_mr, "", "", "point")

# Report a success message    
arcpy.AddMessage("All done!")

##### May 9th, 2022

In [ ]:
# import system modules
import arcpy
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
in_lat = arcpy.GetParameterAsText(0) # Double, Y
in_long = arcpy.GetParameterAsText(1) # Double, X
point_shp = arcpy.GetParameterAsText(2) # feature class
point_buf = arcpy.GetParameterAsText(3) # feature class
point_buf_fl = arcpy.GetParameterAsText(4) # feature layer
intsc_buf_mr = arcpy.GetParameterAsText(5) # feature layer
buffer_dist = arcpy.GetParameterAsText(6) # Linear unit

point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)

arcpy.CopyFeatures_management(point_geom, point_shp)

arcpy.Buffer_analysis(point_shp, point_buf, buffer_dist)

arcpy.MakeFeatureLayer_management(point_buf, point_buf_fl)

arcpy.Intersect_analysis([point_buf_fl, "C:\ArcGIS\Projects\Dev_Proto\Dev_Proto.gdb\MajorRoads"], intsc_buf_mr, "", "", "point")

# Report a success message    
arcpy.AddMessage("All done!")

In [ ]:
# import system modules
import arcpy
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
in_lat = arcpy.GetParameterAsText(0) # Double, Y
in_long = arcpy.GetParameterAsText(1) # Double, X
point_shp = arcpy.GetParameterAsText(2) # feature class
point_buf = arcpy.GetParameterAsText(3) # feature class
point_buf_fl = arcpy.GetParameterAsText(4) # feature layer
buffer_dist = arcpy.GetParameterAsText(5) # Linear unit

point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)

arcpy.CopyFeatures_management(point_geom, point_shp)

arcpy.Buffer_analysis(point_shp, point_buf, buffer_dist)

arcpy.MakeFeatureLayer_management(point_buf, point_buf_fl)

# arcpy.Intersect_analysis([point_buf, ])

# Report a success message    
arcpy.AddMessage("All done!")

In [ ]:
# import system modules
import arcpy
from arcpy import env

# define workspace
arcpy.env.workspace = "C:/ArcGIS/Projects/Dev_Proto"

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
in_lat = arcpy.GetParameterAsText(0) # Double
in_long = arcpy.GetParameterAsText(1) # Double
point_shp = arcpy.GetParameterAsText(2) # feature class
point_buf = arcpy.GetParameterAsText(3) # feature class
point_buf_fl = arcpy.GetParameterAsText(4) # feature layer
buffer_dist = arcpy.GetParameterAsText(5) # Linear unit

ptList =[[in_long, in_lat]]
pt = arcpy.Point()
ptGeoms = []
for p in ptList:
   pt.X = p[0]
   pt.Y = p[1]
   ptGeoms.append(arcpy.PointGeometry(pt))

arcpy.CopyFeatures_management(ptGeoms, point_shp)

arcpy.Buffer_analysis(point_shp, point_buf, buffer_dist)

arcpy.MakeFeatureLayer_management(point_buf, point_buf_fl)

# arcpy.Intersect_analysis([point_buf, ])

# Report a success message    
arcpy.AddMessage("All done!")